In [50]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [36]:
data_df = pd.read_csv('riceClassification.csv')
data_df

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
18180,18181,5853,148.624571,51.029281,0.939210,6008,86.326537,0.498594,332.960,0.663444,2.912535,0
18181,18182,7585,169.593996,58.141659,0.939398,7806,98.272692,0.647461,385.506,0.641362,2.916910,0
18182,18183,6365,154.777085,52.908085,0.939760,6531,90.023162,0.561287,342.253,0.682832,2.925396,0
18183,18184,5960,151.397924,51.474600,0.940427,6189,87.112041,0.492399,343.371,0.635227,2.941216,0


In [37]:
data_df.drop(['id'], axis=1, inplace=True)

In [38]:
data_df.columns

Index(['Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity',
       'ConvexArea', 'EquivDiameter', 'Extent', 'Perimeter', 'Roundness',
       'AspectRation', 'Class'],
      dtype='object')

In [39]:
input_cols = ['Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity',
       'ConvexArea', 'EquivDiameter', 'Extent', 'Perimeter', 'Roundness',
       'AspectRation']
target_cols = 'Class'

In [40]:
train_val_df, test_df = train_test_split(data_df, test_size=0.2)
train_df, val_df = train_test_split(train_val_df, test_size=0.2)

In [41]:
scaler = MinMaxScaler().fit(train_df[input_cols])
train_df[input_cols] = scaler.transform(train_df[input_cols])
val_df[input_cols] = scaler.transform(val_df[input_cols])
test_df[input_cols] = scaler.transform(test_df[input_cols])

In [46]:
x_train = torch.tensor(train_df[input_cols].values, dtype=torch.float32).to(device)
x_val = torch.tensor(val_df[input_cols].values, dtype=torch.float32).to(device)
x_test = torch.tensor(test_df[input_cols].values, dtype=torch.float32).to(device)

In [ ]:
y_train = torch.tensor(train_df[target_cols].values, dtype=torch.float32).to(device)
y_val = torch.tensor(val_df[target_cols].values, dtype=torch.float32).to(device)
y_test = torch.tensor(test_df[target_cols].values, dtype=torch.float32).to(device)

In [51]:
train_ds = TensorDataset(x_train, y_train)
val_ds = TensorDataset(x_val, y_val)
test_ds = TensorDataset(x_test, y_test)

In [52]:
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=True)

In [67]:
len(x_train[1])

10

In [68]:
input_dim = x_train.shape[1]
hidden_layer = 10

In [71]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_layer)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(hidden_layer, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.input_layer(x)
        x = self.relu(x)
        x = self.linear(x)
        x = self.sigmoid(x)
        return x
    
model = MyModel().to(device)

In [73]:
summary(model, input_size = (input_dim,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             110
              ReLU-2                   [-1, 10]               0
            Linear-3                    [-1, 1]              11
           Sigmoid-4                    [-1, 1]               0
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [74]:
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=0.001)

In [76]:
total_loss_training_plot = []
total_loss_validation_plot = []
total_acc_training_plot = []
total_acc_validation_plot = []

In [ ]:
for epoch in range(100):
    total_acc_train = 0
    total_acc_val = 0
    total_loss_train = 0
    total_loss_val = 0\
        
    for data in train_loader:
        
        inputs, labels = data
        
        prediction = model(inputs).squeeze(1)
        
        loss = criterion(prediction, labels)
        
        total_loss_train += loss.item()
        
        acc = ((prediction).round() == labels).sum().item()
        
        total_acc_train += acc
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            
            val_pred = model(inputs).squeeze(1)
            
            loss = criterion(val_pred, labels)
            
            total_loss_val += loss.item()

            acc = ((val_pred).round() == labels).sum().item()

            total_acc_val += acc
        
    total_loss_training_plot.append(round(total_loss_train / 1000, 4))
    total_loss_validation_plot.append(round(total_loss_val / 1000, 4))
    total_acc_training_plot.append(round(total_acc_train / train_ds.__len__() * 100, 4))
    total_acc_validation_plot.append(round(total_loss_val / val_ds.__len__() * 100, 4))

    
    print(f'''Epoch no. {epoch + 1} Train Loss: {total_loss_train/1000:.4f} Train Accuracy: {(total_acc_train/(train_ds.__len__())*100):.4f} Validation Loss: {total_loss_val/1000:.4f} Validation Accuracy: {(total_acc_val/(val_ds.__len__())*100):.4f}''')
    print("="*50)
     

Epoch no. 1 Train Loss: 0.0075 Train Accuracy: 98.6510 Validation Loss: 0.0018 Validation Accuracy: 98.6254
Epoch no. 2 Train Loss: 0.0074 Train Accuracy: 98.6424 Validation Loss: 0.0018 Validation Accuracy: 98.7285
Epoch no. 3 Train Loss: 0.0074 Train Accuracy: 98.6596 Validation Loss: 0.0018 Validation Accuracy: 98.7285
Epoch no. 4 Train Loss: 0.0074 Train Accuracy: 98.6080 Validation Loss: 0.0019 Validation Accuracy: 98.6254
Epoch no. 5 Train Loss: 0.0074 Train Accuracy: 98.6166 Validation Loss: 0.0018 Validation Accuracy: 98.7285
Epoch no. 6 Train Loss: 0.0074 Train Accuracy: 98.6166 Validation Loss: 0.0019 Validation Accuracy: 98.6254
Epoch no. 7 Train Loss: 0.0074 Train Accuracy: 98.6510 Validation Loss: 0.0018 Validation Accuracy: 98.6598
Epoch no. 8 Train Loss: 0.0074 Train Accuracy: 98.6424 Validation Loss: 0.0018 Validation Accuracy: 98.7285
Epoch no. 9 Train Loss: 0.0074 Train Accuracy: 98.6080 Validation Loss: 0.0019 Validation Accuracy: 98.5911
Epoch no. 10 Train Loss: 0.0

In [89]:
with torch.no_grad():
    total_loss_test = 0
    total_acc_test = 0
    
    for data in test_loader:
        inputs, labels = data
        
        test_pred = model(inputs).squeeze(1)
        
        loss = criterion(test_pred, labels)
        
        total_loss_test += loss.item()
        
        acc = ((test_pred).round() == labels).sum().item()
        
        total_acc_test += acc        
accuracy = total_acc_test / len(test_ds) * 100
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 98.71%
